In [23]:
import requests
import base64

client_id = "e0fe32674df742409f8df99d22e1c377"
client_secret = "c3719ea3b1944abb9bc8894acdb4e8a2"

# Codifica credenciais em Base64
auth_str = f"{client_id}:{client_secret}"
b64_auth_str = base64.b64encode(auth_str.encode()).decode()

# Faz o POST para obter o token
url = "https://accounts.spotify.com/api/token"
headers = {
    "Authorization": f"Basic {b64_auth_str}",
}
data = {
    "grant_type": "client_credentials"
}

response = requests.post(url, headers=headers, data=data)
token = response.json()["access_token"]

print("Token de acesso:", token)

Token de acesso: BQD5gIsjsMaTcJe4oreSnpXnoy11lm65trSOD4ru62hyX78IczDpysX_kbrf9vVVe0ieURJWoryQIK70h3kk3ur4l3sQU5Uww8ppxVKJTCsUntB3ee50sf0Q7Dp8PnUbSzY5794JbYY


In [ ]:

import pandas as pd

headers = {"Authorization": f"Bearer {token}"}

playlists = {
    "Top 50 Mundo": "0HGKGYR7s2hhPSqsMIkyQa",
    "Top 50 Brasil": "5sUSFDXe9CWnVqIrwywKJN",
    "Top 70, 80 e 90": "43GQoV2sy4Lr1xpbGQDoHz",
    "Top anos 2000": "6qp5KEHatZ82eBnQ60xcZC",
    "Top anos 2010": "1ailkuppKuq8cfEdZBhuLi",
}

tracks = []

for nome_playlist, playlist_id in playlists.items():
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}/tracks?limit=100"
    r = requests.get(url, headers=headers)

    if r.status_code != 200:
        print(f"Erro ao buscar playlist {nome_playlist} ({playlist_id}):", r.status_code, r.text)
        continue

    data = r.json()
    items = data.get("items")
    if not items:
        print(f"Nenhum 'items' retornado para {nome_playlist}: {data}")
        continue

    for item in items:
        track = item.get("track")
        if track:
            tracks.append({
                "playlist": nome_playlist,
                "track_id": track.get("id"),
                "track_name": track.get("name"),
                "artist": track.get("artists", [{}])[0].get("name"),
                "album": track.get("album", {}).get("name"),
                "popularity": track.get("popularity"),
                "duration_ms": track.get("duration_ms")
            })

df_tracks = pd.DataFrame(tracks)
print("Total de faixas:", len(df_tracks))
df_tracks.head(30)


In [27]:
# gere uma contagem por playlist do df df_tracks

df_tracks['playlist'].value_counts()

playlist
Top 70, 80 e 90    100
Top anos 2010       70
Top anos 2000       70
Top 50 Mundo        50
Top 50 Brasil       50
Name: count, dtype: int64

In [ ]:
def get_audio_features(track_ids):
    # A API aceita até 100 IDs por chamada
    url = "https://api.spotify.com/v1/audio-features"
    ids = ",".join(track_ids)
    r = requests.get(f"{url}?ids={ids}", headers=headers)
    return r.json()["audio_features"]

# Divide em blocos de até 100 músicas
audio_features = []
for i in range(0, len(df_tracks), 100):
    batch = df_tracks["track_id"].iloc[i:i+100].tolist()
    feats = get_audio_features(batch)
    audio_features.extend(feats)

df_features = pd.DataFrame(audio_features)
